# Week2 Notes

It's possible to cast a logistic regression as a simple neural network.  This is what we do here, along with introducting the backward pass and forward pass as part of the computation graph. Finally, we introduce a vectorized implementation and demonstrate its improved efficiency.


## Logistic Regression as a Neural Network
Casting logistic regression as a neural network.

### Binary Classification

The problem of Binary classification, is one of finding a mapping from a feature vector $\mathbf{x}$ to a target $y$ - y must be 1 or 0 - from examples in a historical dataset.

The 'hello world' problem in machine learning (at least for computer vision) is one of learning a mapping of an image ($x$) as either having a cat in the image ($y=1$) or not $y=0$.

An image is an array of numbers (from 0 to 255) made up of 3 channels (one for each of Red, Green and Blue) across the height and width of the image. Each number is the intensity of the pixel in each color channel at a given location in the image. We can think of this as a 3 dimensional array, $A$. The way to get a feature vector $x$ from $A$ is to unroll, unravel, or reshape $A$ into a vector. 

For an image that is $64$x$64$ pixels, there are $64$x$64$x$3$$ =1228$ numbers that make up the feature vector. The length of the feature vector is referenced as $n = n_{X}$.

Let's say we have $m = m_{train}$ examples of images with cats and without cats in them. Then we can define the data as a feature matrix $X$, target vector $Y$ as below:

$$
X = \left [ x^{(1)}, \ldots,  x^{(m)} \right ]
$$
$$
Y = \left [ y^{(1)}, \ldots,  y^{(m)} \right ]
$$

In this setting `X` is a matrix with `X.shape = (m, n)`, and `Y` is a row vector with `Y.shape = (1, m)`. This mean that the individual $x^{(i)}$ are column vectors.

### Logistic Regression

In the logistic regression model, we specify that a linear combination of the features, $x$, as inputs to a non-linear function, in this case the sigmoid function. It produces an estimate of the chance that $y$ is 1, $P(y=1|x) = \hat{y}$, as opposed to explicitly trying to model the target as as function of $x$.

If we tried to model the target, we can't get a smooth function that maps to 1 or 0 only. By using the sigmoid we ensure that the probability of $y$ being 1 is explicitly modelled, and bounded between 0 and 1. The sigmoid function is:

$$
\sigma(z) = \left( 1 + e^{-z} \right)^{-1}
$$


We will see later, that for hidden layers it might make sense to allow the non-linear function, also called the activation function, to be a hyperparameter.

If the linear combination is a large number, then $P(y=1|x)$ gets very close to 1. Using a certain framework (Maximum A Prior or MAP) we can predict that in such cases y will be 1.

If the linear combination is a large negative number, then  $P(y=1|x)$ gets very close to 0. Using the MAP framework we can predict that in such cases y will be 1.
 
If the linear combination is close to 0, then $P(y=1|x)$ is close to 0.5. Under MAP we can make a prediction either side of 0.5 (> 0.5 to 1, <0.5 to 0), but we can't be very sure about which it is.

#### with explicit bias `b`

Using parameters $\mathbf{w} = [w_1, \ldots, w_n]$ (a column vector) and b, a scalar bias parameter. We model $\hat{y}$ as $\hat{y} = \sigma(w^{T}x + b)$.

#### with implicit bias $\theta_0$

If we extend x by adding $x_0=1$, set an example feature as $x = (x_0, \ldots, x_n)$ and define $(w, b) = \mathbf{\theta}$ - still a column vector, with $\theta_0 = b$ then it we can write this model as:

$\hat{y} = \sigma(\mathbf{\theta}^{T}x)$

This formulation is not preferred because by having the bias term explicitly available, some of the future arithmetic becomes easier to understand.

### Logistic Regression Cost Function

It is desirable that as we learn the parameters of the logistic regression that for each of the examples in our dataset, we try for each example $i$, to make $\hat{y^{(i)}}$ as close to $y^{(i)}$ as possible. We can measure the loss, or distance, of each estimate $\hat{y^{(i)}}$ from the ideal target $y^{(i)}$ using a function $L$. Some candidate $L$ functions are:

1. $L(y, \hat{y}) = |y - \hat{y}|$

1. $L(y, \hat{y}) = \frac{1}{2}(y - \hat{y})^2$

1. $L(y, \hat{y}) = - \left [ y\log(\hat{y}) + (1-y)\log(1-\hat{y}) \right ]$

Of these, the third loss function has many desirable properties such as: 

* differentiable, smooth and convex with unique optima (can be confirmed by taking second derivative wrt parameters)
* a natural mapping to the negative log-likelihood (discussed later)

The first is the most compelling, later we will explain second part - the negative log-likelihood. So we can change the parameters $w$ and $b$ until we reach a low loss.

Since we want to reduce the loss for all the dataset, then we can use the individual losses to define an averaged loss function over all examples. This is called the cost function J of the dataset. That is:

$$
J(\mathbf{w}, b) = \frac{1}{m}\sum_{i=1}^{m} L(y^{(i)}, \hat{y^{(i)}})
=  \frac{-1}{m}\sum_{i=1}^{m}  \left [ y\log(\hat{y}) + (1-y)\log(1-\hat{y}) \right ]
$$

Because of our choice of loss function (convex and smooth), we can guarantee that J has a single global optima. 

By iterating using a Taylor series approximation (known as gradient descent) we converge to the optimal parameters that minimize the cost function. 

These parameters define a classification rule, or classifier to model the binary data.

### Gradient Descent

#### Deriving The Algorithm

##### Derivation from Taylor's Therorem

By Taylor's theorem we note that for a well behaved smooth multivariate function F:

$$
F(\mathbf{x} + \mathbf{h}) \approx  F(\mathbf{x}) + \mathbf{h} \cdot \nabla F(\mathbf{x}) + \mathbf{h} \cdot \nabla^2 F(\mathbf{x}) \cdot \mathbf{h}^T
$$

We can rewrite this with the substitution, $\mathbf{x}_t = \mathbf{x} + \mathbf{h}$ and 
$\mathbf{x}_{t-1} = \mathbf{x}$:

$$
F(\mathbf{x}_t) \approx  F(\mathbf{x}_{t-1}) + (\mathbf{x}_{t} -\mathbf{x}_{t-1}) \cdot \nabla F(\mathbf{x}_{t-1}) + (\mathbf{x}_{t} -\mathbf{x}_{t-1}) \cdot \nabla^2 F(\mathbf{x}_{t-1}) \cdot (\mathbf{x}_{t} -\mathbf{x}_{t-1})^T
$$


Near an optima, $\mathbf{x}_{opt}$, $\nabla F(\mathbf{x}_{opt}) = \mathbf{0}$. So if we say that $\mathbf{x}_{t}$ is close to the optima:

$$
F(\mathbf{x}_t) \approx  F(\mathbf{x}_{t-1}) +  (\mathbf{x}_{t} -\mathbf{x}_{t-1}) \cdot \nabla^2 F(\mathbf{x}_{t-1}) \cdot (\mathbf{x}_{t} -\mathbf{x}_{t-1})^T
$$


Rearranging, and noting that $-(\mathbf{x}_{t} -\mathbf{x}_{t-1})^{-1}(F(\mathbf{x}_t) -  F(\mathbf{x}_{t-1})) \approx  - \nabla F(\mathbf{x}_{t-1})$, we have:

$$
 - \nabla F(\mathbf{x}_{t-1}) \approx \nabla^2 F(\mathbf{x}_{t-1}) \cdot (\mathbf{x}_{t} -\mathbf{x}_{t-1})^T
$$

And taking another approximation to the inverse hessian, by a constant (can check dimensions on array multiplications to see that this works):

$$
- \alpha  \nabla F(\mathbf{x}_{t-1}) \approx\ (\mathbf{x}_{t} -\mathbf{x}_{t-1})
$$

Then rearrraning, we have the famous gradient descent algorithm:

From a nearby good starting point $x_{0}$ , and small enough $\alpha$, if we repeat this iteration enough times:

$$
\mathbf{x}_{t}  \approx  \mathbf{x}_{t-1}  - \alpha \nabla F(\mathbf{x}_{t-1})
$$

Then for some R, All $T > R$ will have $x_{T}$ being arbitrarily close to $x_{opt}$.

##### Derivation From Gradient Defintion

The definition of the gradient at a point is the instantaneous slope at that point. This gradient is the fastest direction on the surface where the function is increasing.

So if we want to decrease the function, the best direction is the opposite of the gradient by some small enough amount ($\alpha$) where the gradient approximation is close to the function.  The tension is setting $\alpha$ small enough to be valid but large enough to make progress.

For the 1 dimensional case: If the gradient is positive, then the function is increasing, and we should go in the opposite direction (and increase our point) to reduce the function. If the gradient is negative then we should go in the opposite direction to follow it to a lower value (and reduce our point).

Using this logic, is how we get:

$$
\mathbf{x}_{t}  \approx  \mathbf{x}_{t-1}  - \alpha \nabla F(\mathbf{x}_{t-1})
$$

#### Cost Function

With our logistic function, $J$ is a convex surface over the parameter space. Because of this convexity, we can initialize our parameters anywhere in the space, conventionally the zero vector is chosen for w and 0 for b.

Now we do our iterative process 


### Derivatives

Here Professor Ng provides an understanding of the gradient as a rate of change, using simple analytic examples with numbers. Essentially the point being driven home is:

$$
F(a + h) - F(a) \approx h \frac{\partial F}{\partial x} |_{@x=a}
$$

### More Derivatives Examples
More of the same, some more complex examples with non-constant gradients.


### Computation Graph

Here Professor Ng shows the composite expressions defining an expression can be used to evaluate a function. The major reason this is useful is for caching results,  the forward and the backward propogation of the neural network algorithm. Not that:

- Forward propogation: Substituting intermediate quantities to calculate cost function. 
- Backward propogation: Substituting chain rule (intermediate derivatives) to calculate derivatives of cost function with respect to parameters.

### Derivatives with a Computation Graph

A detailed example of derivatives using the chain rule, numerical examples on a simple compound function. The substitutions can be viewed on the computation graph - a directed acyclic graph.

In code, we note the convention that for a function that is principal and being optimized, say $J$. we denote its derivative wrt some variable $a$ $\frac{\delta J}{da}$ as `da`. This avoids repeatedly referencing the function to be optimized in lengthy token names such as `dJ_over_da`.


### Logistic Regression Gradient Descent

Here we calculate the intermediate derivative quantities needed to complete the chain rule. That is derivatives of the composition functions, to get the chain-ruled derivative of the loss function with respect to the parameters $\mathbf{w}$ and b.

Firstly note that the loss function can be expressed as a composite of:

- $z(w,b;t) = w^Tt +b$

- $\sigma(t) = g(t)= (1 + e^{-t})^{-1}$

- $a(t) = \hat{y}^{(t)} = g(t) = \sigma(t)$

- $e(t, r) = t\log(r)$

- $L(t, r) = -e(t, r) - e(1-t, 1-r)$

In fact we can see that $L$ is:

$$L(y,\hat{y}) = - \left[ e(y, \sigma(z(w,b;x)) + e(1-y, 1 - \sigma(z(w,b;x)) \right ]$$

The derivatives of these functions are:

$ \frac{\partial}{\partial w} z(w,b;t) = t $

$ \frac{\partial}{\partial b} z(w,b;t) = \mathbf{1} $

$ \frac{\partial}{\partial t} \sigma(t) = \sigma(t)(1 - \sigma(t)) $

$ \frac{\partial}{\partial t} \hat{y}^{(t)} = \hat{y}^{(t)}(1 - \hat{y}^{(t)}) $

$ \frac{\partial}{\partial r} e(t, r) = \frac{t}{r}$

$ \frac{\partial}{\partial r} e(1 - t, 1 - r) = -\frac{1-t}{1-r}$

$ \frac{\partial}{\partial r} L(t, r) = - \left( \frac{t}{r} + -\frac{1-t}{1-r} \right ) = \frac{r-t}{r(1-r)}
$

#### Derivative Of The Loss Function

Now to compute the derivative of the loss function wrt to $w$ and $b$, for an example which has feature $\mathbf{x}$ and target $y$:

$$
\frac{\partial}{\partial w} L(y,\hat{y}) = \frac{\partial L}{\partial r} \cdot \frac{\partial y}{\partial \sigma} \cdot \frac{\partial \sigma}{\partial z} \cdot \frac{\partial z}{\partial w} = (\hat{y} - y) \mathbf{x} = (a - y) \mathbf{x}
$$

$$
\frac{\partial}{\partial b} L(y,\hat{y}) = \frac{\partial L}{\partial r} \cdot \frac{\partial y}{\partial \sigma} \cdot \frac{\partial \sigma}{\partial z} \cdot \frac{\partial z}{\partial b} = (\hat{y} -y) = (a-y)
$$

### Gradient Descent on `m` Examples

Now all that remains is to combine the results from above for the cost function, which is just an average of the individual losses across the dataset. Since that's just a linear combination of the individual losses, the derivative of the cost function function will be a linear combination of the derivatives of the individual losses. 

#### Derivative of Cost Function

$$
\frac{\partial}{\partial w} J(w, b; X, Y)  = \frac{1}{m} \sum_i (a^{(i)} -y^{(i)}) \mathbf{x}^{(i)}
$$

$$
\frac{\partial}{\partial b} J(w, b; X, Y)  = \frac{1}{m} \sum_i (a^{(i)} -y^{(i)})
$$

#### Gradient Descent on Cost Function

We are given starting point, $(\mathbf{w}_0,b_0)$ usually chosen as the zero vector and zero for logistic regression, and  some sequence $\alpha_{k}$ (maybe constant), convergence criteria $\epsilon$ and some max number of iterations. 

Run the follow iteration until convergence or the maximum number of iterations allowed:

$ \mathbf{w}_{k} = \mathbf{w}_{k-1} - \alpha_{k} \frac{1}{m} \sum_i (a^{(i)} - y^{(i)})\mathbf{x}^{(i)} $

$ b_{k} = b_{k-1} - \alpha_{k} \frac{1}{m} \sum_i (a^{(i)} -y^{(i)}) $

Remember that for all $i$,  $a^{(i)}$ depend on $w_{k-1}$ and $b_{k-1}$ as well as $i$.

We'll now implement this as `python` unvectorized code.

In [6]:
import math

def z(w, b, x):
    res = b
    for i in range(len(w)):
        res +=w[i]*x[i]
    return res

def sigma(z):
    return (1 + math.exp(-z))**(-1)

def a(w, b, x):
    return sigma(z(w,b,x))

def gradient_of_cost_fn(X, Y, w, b):
    n = len(X)
    m = len(X[0])
    for j in range(m):
        z = z(w, b, X[:][j])
        a = sigma(z)
        dz = a - Y[j]
        for i in range(n):
            dw[j] += dz * X[i][j]
        db += dz
    return (dw, db)

def unvectorized_gradient_descent(X, Y, gradient_fn, w0 = None, b0 = None, 
                                  alpha = 0.001, epsilon = 1e-6, max_iter= 1000):
    """
    
    """
    n = len(X)
    m = len(X[0])
    if w0 is None:
        w0 = [0 for i in range(n)]
    if b0 is None:
        b0 = 0
    w_old = w0
    b_old = b0
    iter_count = 1
    while True:
        dw, db = gradient_of_cost_fn(X, Y, w_old, b_old)
        w_new -= alpha * dw
        b_new -= alpha * db 
        if iter_count > max_iter or (abs(w_new - w_old) < epsilon and abs(b_new - b_old) < epsilon):
            break
        iter_count += 1
        w_old, b_old = w_new, b_new
    return (w_new, b_new)
    

This implementation should work, however it will be super slow.

## NumPy and Vectorization
Python is great easy to learn language. However, because of this easy of use, many data structures such as tuples and lists involve linked lists and type checking. For millions of numbers, as occur in machine learning, this can be a heavy bottleneck.

`numpy` is a python module which defines its own array-like data structure, which is highly optimized by setting a single type (avoids type checking overhead) and storing data in contingous blocks of memory.

### Vectorization

We'll do a comparison of vectorized vs unvectorized code - both with numpy arrays. The main difference illustrated here is that vectorization involves significant speed up because loops is pushed down into the numpy inteface and done using single instruction multiple data (SIMD) format.

In [7]:
import numpy as np

In [8]:
w = np.random.random((int(1e6), 1))
z = np.random.random((int(1e6), 1))

In [9]:
import time

In [10]:
# unvectorized dot product
tic = time.time()
res = 0
for i in range(w.shape[0]):
    res += w[i] * z[i]
toc = time.time()
print(f'Unvectorized result: {res}')
print(f'Time taken for unvectorized calculation {str(100 * (toc - tic))} ms')

Unvectorized result: [249561.16873465]
Time taken for unvectorized calculation 225.06029605865479 ms


In [11]:
# vectorized dot product
# unvectorized dot product
tic = time.time()
res = np.dot(w.T, z)
toc = time.time()
print(f'Vectorized result: {res}')
print(f'Time taken for vectorized calculation {str(100 * (toc - tic))} ms')

Vectorized result: [[249561.16873466]]
Time taken for vectorized calculation 1.2671947479248047 ms


About 2000 times faster to use vectorized result. That's the difference of 1 second or 33 minutes for a calculation to run.

### More Vectorization Examples

In [15]:
# unvectorized: matrix-vector matrix product
n, m = int(1e3), int(1e3)
A = np.random.random((n, m))
x = np.random.random((m, 1))
tic = time.time()
res = np.zeros((m, 1))
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        res[i] += A[i,j] * x[j]
toc = time.time()
#print(f'unvectorized result: {res}')
print(f'Time taken for unvectorized calculation {str(100 * (toc - tic))} ms')

Time taken for unvectorized calculation 373.86600971221924 ms


In [14]:
# vectorized: matrix-vector matrix product
tic = time.time()
res = np.dot(A, x)
toc = time.time()
#print(f'unvectorized result: {res}')
print(f'Time taken for unvectorized calculation {str(100 * (toc - tic))} ms')

Time taken for unvectorized calculation 0.07259845733642578 ms


In [72]:
# higher order function to time vectorized and unvectorized calculations
def timing_vectorization(fn, x = None):
    if x is None: 
        x = np.random.random((int(1e6),1))
    tic = time.time()
    res = 0
    for i in range(x.shape[0]):
        res +=  eval(f'np.{fn}(x[i])')
    toc = time.time()
    #print(f'Unvectorized result: {res}')
    unvectored_diff = 100 * (toc - tic)
    print(f'Time taken for unvectorized calculation {str(unvectored_diff)} ms')
    tic = time.time()
    res = eval(f'np.{fn}(x)')
    toc = time.time()
    vectored_diff = 100* (toc - tic)
    print(f'Time taken for unvectorized calculation {str(vectored_diff)} ms')
    print(f'Ratio of vectored to unvectored calculation time: {str(round(unvectored_diff/vectored_diff, 5))}')

In [73]:
# exp
timing_vectorization('exp', x)

Time taken for unvectorized calculation 1.5041112899780273 ms
Time taken for unvectorized calculation 0.0072956085205078125 ms
Ratio of vectored to unvectored calculation time: 206.16667


In [74]:
# log
timing_vectorization('log', x)

Time taken for unvectorized calculation 1.5859127044677734 ms
Time taken for unvectorized calculation 0.0036954879760742188 ms
Ratio of vectored to unvectored calculation time: 429.14839


In [75]:
# max
timing_vectorization('max', x)

Time taken for unvectorized calculation 1.8209218978881836 ms
Time taken for unvectorized calculation 0.006318092346191406 ms
Ratio of vectored to unvectored calculation time: 288.20755


In [76]:
# min
timing_vectorization('min', x)

Time taken for unvectorized calculation 1.8707990646362305 ms
Time taken for unvectorized calculation 0.0054836273193359375 ms
Ratio of vectored to unvectored calculation time: 341.16087


In [77]:
# abs
timing_vectorization('abs', x)

Time taken for unvectorized calculation 1.4233112335205078 ms
Time taken for unvectorized calculation 0.0041961669921875 ms
Ratio of vectored to unvectored calculation time: 339.19318


In [78]:
# reciprocal
timing_vectorization('reciprocal', x)

Time taken for unvectorized calculation 1.5565156936645508 ms
Time taken for unvectorized calculation 0.0030040740966796875 ms
Ratio of vectored to unvectored calculation time: 518.13492


In [79]:
# square
timing_vectorization('square', x)

Time taken for unvectorized calculation 1.5377998352050781 ms
Time taken for unvectorized calculation 0.0032901763916015625 ms
Ratio of vectored to unvectored calculation time: 467.3913


### Vectorizing Logistic Regression

Let's work through the calculations earlier for backward and forward propogation to get a faster optimization algorithm. As before, we have feature matrix $X$ and target vector $Y$. 

#### Forward Propogation

$$ \hat{Y} = A =  \sigma(wX + b \mathbf{1}^T) $$

#### Backward Propogation

$ dZ = A - Y $ is an intermediate quantity calculated from above. `dZ.shape = (m, 1)`. This helps to calculates the gradient with respect to the parameters.

$d\mathbf{w} = \frac{X(dZ)^T}{m}$ the vector of weights (`dW.shape = (n,1)`)

$db = \frac{\mathbf{1}(dZ)^T}{m} = \frac{ \sum_i (dZ)_i}{m}$, the scalar bias


These can then be used in the gradient descent algorithm

### Broadcasting in Numpy

In numpy, broadcasting expands vectors/matrices so that binary calculations can be confirmed. There are three rules to this.

>
> * Rule 1: If the two arrays differ in their number of dimensions, the shape of the one with >fewer dimensions is padded with ones on its leading (left) side.
>
> * Rule 2: If the shape of the two arrays does not match in any dimension, the array with >shape equal to 1 in that dimension is stretched to match the other shape.
>
> * Rule 3: If in any dimension the sizes disagree and neither is equal to 1, an error is raised.
 >
 from https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html

In [81]:
x.shape  # column vector

(1000, 1)

In [83]:
# broadcasing the constant into a column vector and then addition applied
(x + 1).shape

(1000, 1)

### A Note on Numpy Vectors

One needs to take care with vectors that are 1d. A vector x is 1d if: `x.shape = (1,)`. Such vectors are neither row vectors (`row_vector.shape = (n,1)`) or column vectors (`column_vector.shape = (1,m)`) and exhibit counter-intuitive behaviour. For example, applying a transpose doesn't change the shape of a 1d vector. 

Care should be taken to avoid such vectors as much as possible. One way is to provide `keepdims=True` argument to operations that would otherwise collapse dimensions into a 1d vector.



### Explanation of Logistic Regression Cost Function

The logistic regression cost function, $J$, is the negative log-likelihood (nll) of assuming that the data are bernoulli observations of a varying probability which changes with each example. A sigmoid transform of a linear combination of the features estimates the probability of each example being $1$ or $0$. 

This is a discriminative model, making no assumptions about the distribution of the features.

Formally:

$ P(y | x) = p^y (1-p)^{(1-y)}$

where p is estimated by $\hat{y}$